<a href="https://colab.research.google.com/github/minhmax098/U-Net-LUNA16/blob/main/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation

In [24]:
!pwd

/content/drive/MyDrive/LUNA16/Lung-nodule-detection-LUNA-16


In [23]:
import os
os.chdir('/content/drive/MyDrive/LUNA16/Lung-nodule-detection-LUNA-16')

In [2]:
!pip install SimpleITK
!pip install segmentation-models-pytorch
%matplotlib inline
import SimpleITK as sitk
import numpy as np
import csv
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.6 MB/s eta 0:00:00


In [28]:
import segmentation_models_pytorch as smp
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import tqdm
import time
from torch.utils import data
import os
import torch.nn
import torch.nn.functional as F
from torch.autograd import  Variable
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from sklearn.metrics import confusion_matrix
import cv2

## 1. UNET


In [25]:
unet_path = "Results/Unet/Adam_1e-4_ep100_CE+Lov/"

n_classes = 2
net_in_channels = 1

net = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=net_in_channels,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=n_classes,                      # model output channels (number of classes in your dataset)
)

net.load_state_dict(torch.load(unet_path + 'sumnet_best.pt'))
if torch.cuda.is_available():
    net = net.cuda()
net.eval()

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [26]:
apply_norm = transforms.Normalize([-464.62011798179316],[444.2257740178454])

In [29]:
cand_path = "./dataset/candidates.csv"
b_sz = 8
df_node = pd.read_csv(cand_path)
subset = ['3']#,'5']
running_correct = 0
count = 0

orig_list = []
pred_list = []
for s in subset:
    print('Subset:',s)
    luna_subset_path = './dataset/subset'+str(s)+'/'
    all_files = os.listdir(luna_subset_path)
    mhd_files = []
    for f in all_files:
        if '.mhd' in f:
            mhd_files.append(f)
    count = 0
    for m in tq(mhd_files):
        mini_df = df_node[df_node["seriesuid"]==m[:-4]]
        itk_img = sitk.ReadImage(luna_subset_path+m)
        img_array = sitk.GetArrayFromImage(itk_img)
        origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
        spacing = np.array(itk_img.GetSpacing())
        slice_list = []
        if len(mini_df)>0:
            for i in range(len(mini_df)):
                fName = mini_df['seriesuid'].values[i]
                z_coord = mini_df['coordZ'].values[i]
                orig_class = mini_df['class'].values[i]
                pred = 0
                v_center =np.rint((z_coord-origin[2])/spacing[2])
                img_slice = img_array[int(v_center)]
                mid_mean = img_slice[100:400,100:400].mean()
                img_slice[img_slice==img_slice.min()] = mid_mean
                img_slice[img_slice==img_slice.max()] = mid_mean
                img_slice_tensor = torch.from_numpy(img_slice).unsqueeze(0).float()
                img_slice_norm = apply_norm(img_slice_tensor).unsqueeze(0)

                out = F.softmax(net(img_slice_norm.cuda()),dim=1)
                out_np = np.asarray(out[0,1].squeeze(0).detach().cpu().numpy()*255,dtype=np.uint8)

                ret, thresh = cv2.threshold(out_np,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                connectivity = 4
                output = cv2.connectedComponentsWithStats(thresh, connectivity, cv2.CV_32S)
                stats = output[2]
                temp = stats[1:, cv2.CC_STAT_AREA]
                if len(temp)>0:
                    largest_label = 1 + np.argmax(temp)
                    areas = stats[1:, cv2.CC_STAT_AREA]
                    max_area = np.max(areas)
                    if max_area>150:
                        pred = 1
                if pred == orig_class:
                    running_correct += 1
                pred_list.append(pred)
                orig_list.append(orig_class)
                count += 1

Subset: 3


/tmp/ipython-input-1086045682.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for m in tq(mhd_files):


  0%|          | 0/89 [00:00<?, ?it/s]

In [30]:
print('Accuarcy:',(running_correct/count)*100)
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(orig_list, pred_list)
tn, fp, fn, tp = cf.ravel()

sensitivity = tp/(tp+fn)
print('Sensitivity:',sensitivity)
specificity = tn/(tn+fp)
print('Specificity:',specificity)
precision = tp/(tp+fp)
print('Precision:',precision)

Accuarcy: 36.5191073512252
Sensitivity: 0.8226950354609929
Specificity: 0.3640119180360831
Precision: 0.003322925320118021


## 2. CeNN + UNET

In [4]:
mean = np.array([
    -4.64620118e+02,
    -8.74013723e-04,
    -7.96379932e-04
])

std = np.array([
    4.44225775e+02,
    3.78290289e-03,
    3.83985386e-03
])

In [10]:
cenn_unet_path = "Results/UnetCeNN/Adam_1e-4_ep100_CE+Lov/"

n_classes = 2
net_in_channels = 3

net = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=net_in_channels,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=n_classes,                      # model output channels (number of classes in your dataset)
)

net.load_state_dict(torch.load(cenn_unet_path + 'sumnet_best.pt'))
if torch.cuda.is_available():
    net = net.cuda()
net.eval()

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [6]:
#Code written by Rakshith Sathish
#The work is made public with MIT License

import os
import collections
import torch
import numpy as np
import scipy.misc as m
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms


from torch.utils import data


# Corrected lunaLoader (NO PIL for images)

class cennLunaLoader(data.Dataset):
    def __init__(self, split="train", is_transform=True, img_size=512, mean=None, std=None):

        self.split = split
        self.path = "./data/" + split
        self.is_transform = is_transform
        self.img_size = img_size

        self.files = os.listdir(self.path + '/merge_images/')

        if mean is None or std is None:
            raise ValueError("You must pass mean and std (per channel)")

        # Transforms for images AFTER converted to tensor
        self.img_tf = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.Normalize(mean, std)
        ])

        # Label transform
        self.label_tf = transforms.Resize((img_size, img_size), interpolation=0)


    def __len__(self):
        return len(self.files)


    def __getitem__(self, index):
        fname = self.files[index]

        # -----------------------------
        # Load IMAGE (numpy → tensor)
        # -----------------------------
        img_np = np.load(self.path + '/merge_images/' + fname)   # shape (3,512,512)

        if img_np.shape[0] == 3:     # (C,H,W) → (H,W,C)
            img_np = np.transpose(img_np, (1, 2, 0))

        img = torch.from_numpy(img_np).float()        # (H,W,C)
        img = img.permute(2, 0, 1)                    # (C,H,W)

        # -----------------------------
        # Load LABEL
        # -----------------------------
        im_id = fname.split('_')[1]
        lbl_np = np.load(self.path + '/labels/masks_' + im_id)

        label = torch.from_numpy(lbl_np).long().unsqueeze(0)  # (1,H,W)

        # -----------------------------
        # Apply transforms
        # -----------------------------
        if self.is_transform:
            img = self.img_tf(img)
            label = self.label_tf(label)

        return img, label.squeeze(0)


/tmp/ipython-input-4146092141.py:8: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  import scipy.misc as m


In [8]:
# =========================
#  TEST EVALUATION SECTION
# =========================

# 1) Tạo test set (đổi 'test' -> 'val' nếu loader của bạn chưa có split test riêng)
testDset = cennLunaLoader(is_transform=True, split='test', img_size=256, mean=mean, std=std)
testDataLoader = data.DataLoader(testDset,
                                 batch_size=16,
                                 shuffle=False,
                                 num_workers=4,
                                 pin_memory=True)



/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [17]:
from sklearn.metrics import confusion_matrix

use_gpu = torch.cuda.is_available()
dice_list = []
iou_list  = []
sens_list = []
spec_list = []

with torch.no_grad():
    for imgs, mask in tqdm.tqdm(testDataLoader):
        if use_gpu:
            inputs = imgs.cuda()
            labels = mask.cuda()
        else:
            inputs = imgs
            labels = mask

        # forward
        logits = net(inputs)                    # [B, 2, H, W]
        probs  = F.softmax(logits, dim=1)
        preds  = torch.argmax(probs, dim=1)     # [B, H, W], giá trị 0/1

        # p1 = probs[:, 1, :, :]        # [B, H, W]

        # # threshold at 0.8
        # preds = (p1 > 0.5).long()     # [B, H, W] values 0/1

        # tính metric cho từng ảnh trong batch
        for b in range(preds.size(0)):
            y_true = labels[b].cpu().numpy().astype(np.int32)
            y_pred = preds[b].cpu().numpy().astype(np.int32)

            c = confusion_matrix(y_true.ravel(),
                                 y_pred.ravel(),
                                 labels=[0, 1])
            TN, FP, FN, TP = c.ravel()

            dice = (2.0 * TP) / (2.0 * TP + FP + FN + 1e-6)
            iou  = TP / (TP + FP + FN + 1e-6)
            sens = TP / (TP + FN + 1e-6)
            spec = TN / (TN + FP + 1e-6)

            dice_list.append(dice)
            iou_list.append(iou)
            sens_list.append(sens)
            spec_list.append(spec)

# 3) Tính mean ± std để điền vào Table 1
dice_mean = np.mean(dice_list); dice_std = np.std(dice_list)
iou_mean  = np.mean(iou_list);  iou_std  = np.std(iou_list)
sens_mean = np.mean(sens_list)
spec_mean = np.mean(spec_list)

print("=== Test metrics (per-slice) ===")
print(f"Dice mean ± std: {dice_mean:.4f} ± {dice_std:.4f}")
print(f"IoU  mean ± std: {iou_mean:.4f} ± {iou_std:.4f}")
print(f"Sensitivity    : {sens_mean:.4f}")
print(f"Specificity    : {spec_mean:.4f}")


  0%|          | 0/23 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 23/23 [00:07<00:00,  3.24it/s]

=== Test metrics (per-slice) ===
Dice mean ± std: 0.4780 ± 0.3849
IoU  mean ± std: 0.4004 ± 0.3446
Sensitivity    : 0.4837
Specificity    : 0.9993


In [12]:
# Check with train set
trainDset = cennLunaLoader(is_transform=True, split='train', img_size=256, mean=mean, std=std)
trainDataLoader = data.DataLoader(trainDset,
                                 batch_size=16,
                                 shuffle=False,
                                 num_workers=4,
                                 pin_memory=True)
from sklearn.metrics import confusion_matrix

use_gpu = torch.cuda.is_available()
dice_list = []
iou_list  = []
sens_list = []
spec_list = []

with torch.no_grad():
    for imgs, mask in tqdm.tqdm(trainDataLoader):
        if use_gpu:
            inputs = imgs.cuda()
            labels = mask.cuda()
        else:
            inputs = imgs
            labels = mask

        # forward
        logits = net(inputs)                    # [B, 2, H, W]
        probs  = F.softmax(logits, dim=1)
        preds  = torch.argmax(probs, dim=1)     # [B, H, W], giá trị 0/1

        # tính metric cho từng ảnh trong batch
        for b in range(preds.size(0)):
            y_true = labels[b].cpu().numpy().astype(np.int32)
            y_pred = preds[b].cpu().numpy().astype(np.int32)

            c = confusion_matrix(y_true.ravel(),
                                 y_pred.ravel(),
                                 labels=[0, 1])
            TN, FP, FN, TP = c.ravel()

            dice = (2.0 * TP) / (2.0 * TP + FP + FN + 1e-6)
            iou  = TP / (TP + FP + FN + 1e-6)
            sens = TP / (TP + FN + 1e-6)
            spec = TN / (TN + FP + 1e-6)

            dice_list.append(dice)
            iou_list.append(iou)
            sens_list.append(sens)
            spec_list.append(spec)

# 3) Tính mean ± std để điền vào Table 1
dice_mean = np.mean(dice_list); dice_std = np.std(dice_list)
iou_mean  = np.mean(iou_list);  iou_std  = np.std(iou_list)
sens_mean = np.mean(sens_list)
spec_mean = np.mean(spec_list)

print("=== Train metrics (per-slice) ===")
print(f"Dice mean ± std: {dice_mean:.4f} ± {dice_std:.4f}")
print(f"IoU  mean ± std: {iou_mean:.4f} ± {iou_std:.4f}")
print(f"Sensitivity    : {sens_mean:.4f}")
print(f"Specificity    : {spec_mean:.4f}")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 69/69 [04:21<00:00,  3.79s/it]

=== Train metrics (per-slice) ===
Dice mean ± std: 0.8924 ± 0.2495
IoU  mean ± std: 0.8606 ± 0.2448
Sensitivity    : 0.9230
Specificity    : 0.9998


In [ ]:
import cv2

In [ ]:
apply_norm =